In [40]:
# Install necessary libraries quietly so the cell doesn't output logs
!pip install pathway bokeh panel --quiet

In [41]:
# Importing required Python libraries
import pandas as pd     # For data manipulation
import numpy as np      # For numerical operations
import pathway as pw    # Core streaming/dataflow engine
import datetime         # Working with timestamps
import panel as pn      # For layout/dashboard visualizations
import bokeh.plotting   # For creating Bokeh plots

# Initialize Panel extension so Bokeh plots render properly in notebooks
pn.extension()

In [42]:
# Load the dataset (make sure the CSV file is uploaded in Colab)
df = pd.read_csv("/content/dataset.csv")

In [43]:
# Combine 'LastUpdatedDate' and 'LastUpdatedTime' columns into a single datetime object
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'], format='%d-%m-%Y %H:%M:%S')

In [44]:
# Sort the dataset by the newly created timestamp to simulate real-time flow
df = df.sort_values('Timestamp').reset_index(drop=True)

In [45]:
# Preview the data
df.head()

,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime,Timestamp
0,0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,04-10-2016,07:59:00,2016-10-04 07:59:00
1,5248,BHMNCPHST01,1200,26.140014,91.731000,237,bike,low,2,0,04-10-2016,07:59:00,2016-10-04 07:59:00
2,3936,BHMMBMMBX01,687,20.000035,78.000003,264,car,low,2,0,04-10-2016,07:59:00,2016-10-04 07:59:00
3,6560,BHMNCPNST01,485,26.140048,91.730972,249,car,low,2,0,04-10-2016,07:59:00,2016-10-04 07:59:00
4,17056,Shopping,1920,26.150504,91.733531,614,cycle,low,2,0,04-10-2016,07:59:00,2016-10-04 07:59:00


In [46]:
# Save required columns to a new CSV that will be streamed later via Pathway
df[["Timestamp", "SystemCodeNumber", "Capacity", "Occupancy"]].to_csv("parking_stream.csv", index=False)

In [47]:
# Define schema for streaming data
# (Only relevant fields are included for Model 1)\
class ParkingSchema(pw.Schema):
    Timestamp: str
    SystemCodeNumber: str
    # Latitude: float
    # Longitude: float
    Capacity: int
    Occupancy: int
    # QueueLength: int
    # TrafficConditionNearby: float
    # IsSpecialDay: int
    # VehicleType: str

In [48]:
# Load and simulate streaming data using Pathway's demo method
data = pw.demo.replay_csv("parking_stream.csv", schema=ParkingSchema, input_rate=1000)

In [49]:
# # Define the mapping function using pw.udf
# @pw.udf
# def map_vehicle_type(vehicle_type: str) -> float:
#     if vehicle_type == "car":
#         return 1.0
#     elif vehicle_type == "bike":
#         return 0.7
#     elif vehicle_type == "truck":
#         return 1.5
#     else:
#         return 1.0  # default weight

In [50]:
# Define the datetime format string for parsing
fmt = "%Y-%m-%d %H:%M:%S"

# Add new columns to the stream:
# - 't' for parsed datetime
# - 'occ_rate' = occupancy ratio = Occupancy / Capacity
data = data.with_columns(
    t = data.Timestamp.dt.strptime(fmt),
    occ_rate = data.Occupancy / data.Capacity,
    # veh_weight = map_vehicle_type(data.VehicleType)
)

In [51]:
# Linear pricing coefficient — basically how aggressively the price increases with occupancy
alpha = 5

# Create the baseline pricing model
# Price = 10 + α × occupancy_rate
baseline_model = data.with_columns(
    price_baseline = 10 + alpha * data.occ_rate
)

In [52]:
# Define a plotting function that uses Bokeh to visualize price over time for a given lot
def plot_price(source, lot_name):
    fig = bokeh.plotting.figure(
        height=200, width=400,
        title=f"Baseline Linear Pricing – {lot_name}",
        x_axis_type="datetime"
    )
    fig.line("t", "price_baseline", source=source, line_width=2, color="navy")
    fig.circle("t", "price_baseline", source=source, size=6, color="red")
    return fig

# Get unique parking lot IDs (SystemCodeNumber values) to plot each one separately
lot_ids = df['SystemCodeNumber'].unique()

# Create Bokeh plots for each parking lot
plots = []

for lot_id in lot_ids:
    # Filter data for current lot
    filtered = baseline_model.filter(pw.this.SystemCodeNumber == lot_id)
    # Bind data to the plot function
    viz = filtered.plot(lambda src: plot_price(src, lot_id), sorting_col="t")
    plots.append(pn.Column(viz))

# Stack all plots into a vertical dashboard layout
pn.Column(*plots).servable()

Column
    [0] Column
        [0] Column
            [0] Row
                [0] Markdown(str)
                [1] TooltipIcon(value='Table depends o...)
            [1] Bokeh(figure)
    [1] Column
        [0] Column
            [0] Row
                [0] Markdown(str)
                [1] TooltipIcon(value='Table depends o...)
            [1] Bokeh(figure)
    [2] Column
        [0] Column
            [0] Row
                [0] Markdown(str)
                [1] TooltipIcon(value='Table depends o...)
            [1] Bokeh(figure)
    [3] Column
        [0] Column
            [0] Row
                [0] Markdown(str)
                [1] TooltipIcon(value='Table depends o...)
            [1] Bokeh(figure)
    [4] Column
        [0] Column
            [0] Row
                [0] Markdown(str)
                [1] TooltipIcon(value='Table depends o...)
            [1] Bokeh(figure)
    [5] Column
        [0] Column
            [0] Row
                [0] Markdown(str)
                [1] TooltipIcon(value='Table depends o...)
            [1] Bokeh(figure)
    [6] Column
        [0] Column
            [0] Row
                [0] Markdown(str)
                [1] TooltipIcon(value='Table depends o...)
            [1] Bokeh(figure)
    [7] Column
        [0] Column
            [0] Row
                [0] Markdown(str)
                [1] TooltipIcon(value='Table depends o...)
            [1] Bokeh(figure)
    [8] Column
        [0] Column
            [0] Row
                [0] Markdown(str)
                [1] TooltipIcon(value='Table depends o...)
            [1] Bokeh(figure)
    [9] Column
        [0] Column
            [0] Row
                [0] Markdown(str)
                [1] TooltipIcon(value='Table depends o...)
            [1] Bokeh(figure)
    [10] Column
        [0] Column
            [0] Row
                [0] Markdown(str)
                [1] TooltipIcon(value='Table depends o...)
            [1] Bokeh(figure)
    [11] Column
        [0] Column
            [0] Row
                [0] Markdown(str)
                [1] TooltipIcon(value='Table depends o...)
            [1] Bokeh(figure)
    [12] Column
        [0] Column
            [0] Row
                [0] Markdown(str)
                [1] TooltipIcon(value='Table depends o...)
            [1] Bokeh(figure)
    [13] Column
        [0] Column
            [0] Row
                [0] Markdown(str)
                [1] TooltipIcon(value='Table depends o...)
            [1] Bokeh(figure)

In [54]:
# Start Pathway's engine to run everything
%%capture --no-display
pw.run()

Output()